# Introduction to AliCoin:

AliCoin is a simplified cryptocurrency simulation built to demonstrate the fundamental concepts of blockchain technology using proof of work (POW). This project implements a working blockchain model with features such as account management, transaction handling, mining, and a Merkle Tree to secure transactions within blocks.

# Core Components and Their Technical Implementation

## 1. **Blockchain Structure**
The blockchain is implemented as a chain of interconnected blocks, where each block contains:
- A list of validated transactions.
- A reference to the hash of the preceding block (`previous_hash`), ensuring immutability.
- A nonce derived from solving a computationally intensive hash puzzle.
- A Merkle Root, derived from the transaction data, to validate integrity.

## 2. **Genesis Block**
The genesis block serves as the starting point of the blockchain. Unlike other blocks, it has no parent and uses a predefined `previous_hash` of `"0"`. It is dynamically created only when:
- The blockchain is empty.
- There are pending transactions to process.

## 3. **Accounts**
Accounts in AliCoin are modeled as simple objects with the following attributes:
- `id`: A unique identifier for the account.
- `balance`: A numeric representation of funds held.
- `transactions`: A list of transaction objects associated with the account.

Each account interacts with the blockchain by sending and receiving funds via transactions. Balances are updated atomically during transaction validation to prevent race conditions.

## 4. **Transactions**
Transactions are immutable objects that encapsulate:
- `sender` and `recipient`: Unique IDs of the sender and recipient accounts.
- `amount`: The value being transferred.
- `id`: A unique identifier for the transaction.
- `timestamp`: A UNIX timestamp marking when the transaction was created.

Transactions are validated before being added to the pending pool. Validation ensures that the sender has sufficient balance, and any invalid transaction is rejected.

## 5. **Merkle Tree**
The Merkle Tree is used to generate a single cryptographic hash (the Merkle Root) representing all transactions in a block.

## 6. **Miners**
Miners are modeled as specialized accounts with computational resources (`resources`) initilased randomly that simulate their hashing power. Mining efficiency is proportional to the `resources` attribute, influencing the time required to compute a valid hash.

## 7. **Mining and Proof-of-Work**
Mining implements a simplified proof-of-work algorithm where miners search for a `nonce` such that the block's hash is numerically smaller than a predefined target (`difficulty`). The process involves:
1. Constructing the block header, which includes:
   - `previous_hash`
   - `merkle_root`
   - `timestamp`
   - `difficulty`
   - `nonce`
2. Iteratively hashing the block header with different nonce values.
3. Verifying whether the resulting hash satisfies the target condition.

The first miner to find a valid nonce get a reward of 10 Alicoins (10 @), ensuring fair incentives.


In [53]:
import hashlib
from datetime import datetime
from random import randint
import random
import time
from multiprocessing import Process, Value, Queue

class MerkleTree:
    def __init__(self, files):
        # Initialize the MerkleTree with a list of files (data items)
        self.data = files

    def calculate_hashes(self):
        # Compute the Merkle root and all hashes for the given data
        all_hashes = []  # Store all hashes computed during the process
        initial_hashes = []  # Store the initial hashes of the data

        # Generate SHA-256 hash for each file in the data
        for file in self.data:
            sha = hashlib.sha256()
            sha.update(str(file).encode())
            initial_hashes.append(sha.hexdigest())

        def merkle_root(hashes):
            # Recursive function to calculate the Merkle root
            all_hashes.extend(hashes)  # Add the current level's hashes to all_hashes

            if not hashes:
                raise ValueError('Missing required files')  # Ensure data exists

            # If the number of hashes is odd, duplicate the last one
            if len(hashes) % 2 != 0:
                hashes.append(hashes[-1])

            new_hashes = []  # Store the next level's hashes

            # Combine and hash pairs of hashes to form the next level
            for x in [hashes[x:x + 2] for x in range(0, len(hashes), 2)]:
                sha = hashlib.sha256()
                sha.update((str(x[0]) + str(x[1])).encode())
                new_hashes.append(sha.hexdigest())

            # If only one hash remains, it is the Merkle root
            if len(new_hashes) == 1:
                return new_hashes[0]
            else:
                return merkle_root(new_hashes)

        # Compute and return the Merkle root
        root = merkle_root(initial_hashes)
        all_hashes.append(root)
        return root


class Transaction:
    def __init__(self, sender, recipient, amount):
        # Initialize a transaction with sender, recipient, amount, timestamp, and ID
        self.sender = sender
        self.recipient = recipient
        self.amount = amount
        self.id = None  # ID will be set later
        self.timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")  # Record current time

    def __str__(self):
        # String representation of the transaction
        return f"Transaction from {self.sender} to {self.recipient} at {self.timestamp} | amount {self.amount} @ | ID: {self.id}"


class Block:
    def __init__(self, transactions, previous_hash, merkle_root, difficulty, hash, timestamp, nonce):
        # Initialize a block with details like transactions, previous hash, and difficulty
        self.index = None  # Index will be set later
        self.timestamp = timestamp
        self.transactions = transactions
        self.previous_hash = previous_hash
        self.merkle_root = merkle_root
        self.nonce = nonce
        self.difficulty = difficulty
        self.hash = hash

    def __str__(self):
        # String representation of the block
        return f"Block {self.index} | Hash: {self.hash}| Merkle root: {self.merkle_root} | Timestamp: {self.timestamp} | Transations: {len(self.transactions)} | Previous Hash: {self.previous_hash} | Nonce: {self.nonce}"


class Account:
    def __init__(self, balance):
        # Initialize an account with a balance and transaction history
        self.balance = balance
        self.transactions = []
        self.id = None  # ID will be set later
        self.private_key = None  # Reserved for cryptographic keys
        self.public_key = None  # Reserved for cryptographic keys

    def __str__(self):
        # String representation of the account
        return f"Account id: {self.id} | Balance: {self.balance} @ | Number of transactions: {len(self.transactions)}"


class Miner(Account):
    def __init__(self, difficulty, balance=0):
        # Initialize a miner as a specialized account
        super().__init__(balance)
        self.difficulty = difficulty
        self.block = []
        self.resources = randint(1, 10)  # Randomly assign computational resources (1-10)

    def __str__(self):
        # String representation of the miner
        return f"Miner id: {self.id} | Balance: {self.balance} @ | Resources: {self.resources}"


class AliCoin:
    def __init__(self):
        # Initialize the blockchain with accounts, miners, and basic settings
        self.accounts = {}  # Dictionary to store accounts
        self.miners = {}  # Dictionary to store miners
        self.transactions = []  # List to store pending transactions
        self.blockchain = []  # List to store the chain of blocks
        self.difficulty = 0xFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF  # Mining difficulty

    def create_genesis_block(self):
      # Check if the blockchain is empty and there are transactions to be mined
      if not self.blockchain and self.transactions:
          genesis_block = Block(
              self.transactions,  # Include transactions
              "0",  # Previous hash is zero for genesis block
              MerkleTree(self.transactions).calculate_hashes(),  # Compute Merkle root
              self.difficulty,
              "0",  # Genesis block hash is zero initially
              datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
              0  # Nonce is zero for genesis block
          )
          genesis_block.index = len(self.blockchain)
          self.blockchain.append(genesis_block)
          self.transactions = []  # Clear pending transactions
          print("Genesis block created.")
      elif not self.transactions:
          print("No transactions to create a genesis block.")
      else:
          print("Genesis block already exists.")


    def add_block(self, transactions, merkle_root, hash, timestamp, nonce):
        # Add a new block to the blockchain
        difficulty = self.difficulty
        previous_hash = self.blockchain[-1].hash
        new_block = Block(transactions, previous_hash, merkle_root, difficulty, hash, timestamp, nonce)
        new_block.index = len(self.blockchain)
        self.blockchain.append(new_block)
        self.transactions = []  # Clear pending transactions
        print("Block added successfully.")

    def add_miner(self):
        # Create and register a new miner
        new_miner = Miner(self.difficulty)
        new_miner.id = len(self.miners)
        self.miners[new_miner.id] = new_miner
        print(f"Miner added successfully | ID: {new_miner.id} | Balance: {new_miner.balance} @ | Resources (1-10): {new_miner.resources}")

    def mine_block(self):
      # Check if the genesis block needs to be created
      if not self.blockchain:
          self.create_genesis_block()

      if not self.transactions:
          print("No transactions to mine.")
          return

      # Proceed with mining if there are transactions
      found = Value('i', 0)  # Shared value to indicate a successful mine

      def mine(found, miner_id):
          # Each miner attempts to find a valid nonce
          miner = self.miners[miner_id]
          previous_hash = self.blockchain[-1].hash
          merkle_root = MerkleTree(self.transactions).calculate_hashes()
          nonce = 0
          while not found.value:
              # Simulate computational effort based on miner's resources
              time.sleep(random.uniform(0, 0.0001) * (22 - 2 * (miner.resources)))
              timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
              file = str(previous_hash) + str(merkle_root) + str(timestamp) + str(self.difficulty) + str(nonce)
              sha = hashlib.sha256()
              sha.update(str(file).encode())
              result = sha.hexdigest()
              if int(result, 16) < self.difficulty:
                  with found.get_lock():
                      if not found.value:  # Ensure only one miner is rewarded
                          found.value = 10
                          print(f'Nonce found. Miner id: {miner_id}')
                          result_queue.put((nonce, timestamp, miner_id, merkle_root, previous_hash, result))
                  break
              else:
                  nonce += 1

      result_queue = Queue()
      processes = []

      # Start mining processes for all miners
      for i in range(len(self.miners)):
          process = Process(target=mine, args=(found, i))
          processes.append(process)
          process.start()

      # Wait for all processes to complete
      for process in processes:
          process.join()

      # Process the result if mining was successful
      if not result_queue.empty():
          nonce, timestamp, miner_id, merkle_root, previous_hash, hash = result_queue.get()
          self.miners[miner_id].balance += 10  # Reward the miner
          print("Mining simulation complete.")
          print(f"Nonce: {nonce}, Timestamp: {timestamp}, Miner id: {miner_id}")
      self.add_block(self.transactions, merkle_root, hash, timestamp, nonce)

    def add_account(self, balance):
        # Create and register a new account
        new_account = Account(balance)
        new_account.id = len(self.accounts)
        self.accounts[new_account.id] = new_account
        print(f"Account added successfully | ID: {new_account.id} | Balance {new_account.balance} @")

    def get_account(self, account_id):
        # Retrieve account by its ID
        return self.accounts[account_id]

    def get_balance(self, account_id):
        # Retrieve balance of a specific account
        return self.accounts[account_id].balance

    def add_transaction(self, sender_id, recipient_id, amount):
        # Add a new transaction if valid
        sender = self.get_account(sender_id)
        recipient = self.get_account(recipient_id)
        if sender.balance >= amount:
            transaction = Transaction(sender_id, recipient_id, amount)
            transaction.id = len(self.transactions)
            sender.balance -= amount
            recipient.balance += amount
            self.transactions.append(transaction)
            sender.transactions.append(transaction)
            recipient.transactions.append(transaction)
            print("Transaction added successfully.")
            print(transaction)
        else:
            print(f"Transaction failed. Sender balance: {sender.balance} | Amount: {amount}")

    def get_transaction(self, transaction_id):
        # Retrieve a transaction by its ID
        return self.transactions[transaction_id]


In [54]:
# Initiating a new crypto coin
ALC = AliCoin()

# Adding accounts and miners
ALC.add_account(53)
ALC.add_account(325)
ALC.add_miner()
ALC.add_miner()
ALC.add_miner()
ALC.add_miner()

# Trying to mine the genesis block
ALC.mine_block()

Account added successfully | ID: 0 | Balance 53 @
Account added successfully | ID: 1 | Balance 325 @
Miner added successfully | ID: 0 | Balance: 0 @ | Resources (1-10): 9
Miner added successfully | ID: 1 | Balance: 0 @ | Resources (1-10): 8
Miner added successfully | ID: 2 | Balance: 0 @ | Resources (1-10): 8
Miner added successfully | ID: 3 | Balance: 0 @ | Resources (1-10): 5
No transactions to create a genesis block.
No transactions to mine.


In [55]:
# Adding the first transaction
ALC.add_transaction( sender_id= 0, recipient_id= 1, amount = 12)

Transaction added successfully.
Transaction from 0 to 1 at 2024-11-17 22:24:08 | amount 12 @ | ID: 0


In [56]:
# Trying to send more coins than what's available
ALC.add_transaction(sender_id = 0, recipient_id = 1, amount = 455)

Transaction failed. Sender balance: 41 | Amount: 455


In [57]:
# Mining the genesis block
ALC.mine_block()

Genesis block created.
No transactions to mine.


In [58]:
# More transactions
ALC.add_transaction( sender_id= 0, recipient_id= 1, amount = 20)
ALC.add_transaction( sender_id= 1, recipient_id= 0, amount = 42)

# Mining a new block
ALC.mine_block()

Transaction added successfully.
Transaction from 0 to 1 at 2024-11-17 22:24:08 | amount 20 @ | ID: 0
Transaction added successfully.
Transaction from 1 to 0 at 2024-11-17 22:24:08 | amount 42 @ | ID: 1
Nonce found. Miner id: 0
Mining simulation complete.
Nonce: 35388, Timestamp: 2024-11-17 22:24:19, Miner id: 0
Block added successfully.


In [59]:
# The miner's balance should be 10 since he successfuly mined the block
print(f" Successful miner's new balance: {ALC.miners[0].balance} @ | Other miner's balance: {ALC.miners[1].balance} @ ")

 Successful miner's new balance: 10 @ | Other miner's balance: 0 @ 


In [60]:
print(ALC.blockchain[-1])

Block 1 | Hash: 00000afef44524a3008d076390651e640ffe42e3f27ef238be490787a334c8d0| Merkle root: a1da09b5edaec6a71358cc2a1de132b72244f0e261c6d387233225aedd15a649 | Timestamp: 2024-11-17 22:24:19 | Transations: 2 | Previous Hash: 0 | Nonce: 35388
